In [ ]:
import hashlib
import util
from test_framework.key import generate_schnorr_nonce, ECKey, ECPubKey
from test_framework.musig import aggregate_musig_signatures, aggregate_schnorr_nonces, generate_musig_key, sign_musig

# 1.2 Introduction to n-of-n MuSig

* Part 1: Public Key Generation.
* Part 2: Signing.
    * Nonce Aggregation.
    * Signature Aggregation.

In this chapter, we introduce the interactive [MuSig protocol](https://eprint.iacr.org/2018/068.pdf) which allows n-of-n participants to jointly create and spend taproot or tapscript outputs using aggregated schnorr signatures. 

Using a signature aggregation scheme like MuSig has two significant advantages over using Script's `OP_CHECKMULTISIG` and tapscript's `OP_CHECKSIGADD` opcodes:

* **Transaction Size/Fees**: an aggregate MuSig pubkey and signature is indistinguishable from a single-key pubkey and signature, meaning that the transaction size (and required fee) for a multi-key output are the same as for a single-key output.
* **Privacy and Fungibility**: an aggregate MuSig pubkey and signature is indistinguishable from a single-key pubkey and signature, making it impossible for anyone to use the public block chain data to identify where a multi-key scheme has been used.

The MuSig protocol covers both the initial setup (generating an aggregate pubkey for all participants), and the signing protocol (creating a valid signature for the aggregate pubkey). The signing requires multiple rounds of communication between the individual signers.

Bip-schnorr is linear in the nonce points and public keys, which means that public keys, nonces and signatures can be aggregated. A very naive multiparty signature scheme could be achieved by simply summing the individual pubkeys to generate an aggregate pubkey, each participant signing with a shared nonce, and then summing the signatures. Such a scheme would be vulnerable to both the [key cancellation attack](https://tlu.tarilabs.com/cryptography/digital_signatures/introduction_schnorr_signatures.html#key-cancellation-attack) and private key extraction by exploiting weak or known nonces. Countering these attacks is what adds some complexity to the MuSig protocol.

![test](images/musig_intro_0.jpg)

## Part 1: Public Key Generation

To counter the key cancellation attack, each participant's pubkey is _tweaked_ by a _challenge factor,_ which is generated by hashing all the participants' pubkeys together. Doing this ensures that no individual participant (or group of participants) is able to create a pubkey that cancels out other participants' pubkeys.

The challenge factor is unique for each participant, but all challenge factors are based on a hash of all participants' pubkeys.

No interactive round-trips are required in this step. As long as everyone can get all the public keys involved (through communication, a coordinator or offline) then they can compute the challenge factors and the aggregate public key locally.

![test](images/musig_intro_1.jpg)

#### 1.2.1 _Programming Exercise:_ Compute 3-of-3 MuSig public key

In this exercise, we'll use the `generate_musig_key()` function to generate challenge factors for each participant and an aggregate MuSig pubkey.

`generate_musig_key()` takes a list of the participants' keys `generate_musig_key([ECPubKey0, ECPubKey1, ...])` and returns a challenge map and the aggregate pubkey:
* the challenge map contains `ECPubKey_i, challenge_data_i` key - value pairs.
* The aggregate pubkey is an `ECPubKey` object

In [ ]:
# Compute key pairs
privkey0 = ECKey().generate()
privkey1 = ECKey().generate()
privkey2 = ECKey().generate()
pk0 = privkey0.get_pubkey()
pk1 = privkey1.get_pubkey()
pk2 = privkey2.get_pubkey()
pk_v = [pk0, pk1, pk2]

# Compute Key Challenges


# Multiply key pairs by challenge factor
privkey0_c =  # TODO: implement
privkey1_c =  # TODO: implement
privkey2_c =  # TODO: implement
pk0_c =  # TODO: implement
pk1_c =  # TODO: implement
pk2_c =  # TODO: implement


print("Tweaked privkey0 is {}".format(privkey0_c.secret))
print("Tweaked privkey1 is {}".format(privkey1_c.secret))
print("Tweaked privkey2 is {}".format(privkey2_c.secret))

## Part 2: Signing 

### 2a - Nonce Generation

The first step of creating a MuSig signature requires each signer to generate their own nonce and nonce point. The participants then exchange those nonces points and an aggregate nonce point is derived by summing all the nonce points.

The security proof for MuSig requires that nonces are randomly generated and are independent of each other. To ensure that no individual participant (or group of participants) can create their nonce as a function of the other nonces or individually control what the aggregate nonce point will be, there is an initial round of exchanging hash commitments to the individual nonce points.

Individual participants should only exchange their nonce point when they have received all commitments, and only proceed with signing if all nonce points match their commitments.

Finally, if the aggregate nonce is not a quadratic residue, then it is negated and all individual nonce points are also negated.

![test](images/musig_intro_2.jpg)

#### 1.2.2 _Programming Exercise:_ Compute 3-of-3 MuSig nonce

In this exercise, we'll generate nonces for individual participants, calculate the nonce point commitments, and then generate an aggregate nonce point.

* Use `generate_schnorr_nonce()` to generate a random nonce. The function returns an `ECkey` object which is a valid schnorr nonce (quadratic residue y-value).
* Use `aggregate_schnorr_nonces()` to aggregate those individual nonces. The function takes a list of `ECKey` nonces and returns a the aggregate nonce and a negation flag:
  * the aggregate nonce is an `ECPubKey` object
  * the negation flag is a boolean and indicates that the individual nonces should be negated

In [ ]:
# Generate nonces.
k0 =  # TODO: implement
k1 =  # TODO: implement
k2 =  # TODO: implement

# Nonce point commitment round.
R0_digest = hashlib.sha256(R0.get_bytes()).digest()
R1_digest = hashlib.sha256(R1.get_bytes()).digest()
R2_digest = hashlib.sha256(R2.get_bytes()).digest()

# Aggregate nonces.
R_agg = # TODO: implement

print("Individual nonces are {}, {}, {}.".format(k0.secret, k1.secret, k2.secret))
print("Aggregate nonce point is {}".format(R_agg.get_bytes().hex()))

### 2b - Signature Aggregation

Once all participants have their individual nonces and the aggregate nonce point, then can all sign individually. 

The individual `s` values are then exchanged and summed together. The aggregate `s` value and aggregate nonce point `R` form a valid bip-schnorr signature for the aggregate pubkey.

Notice that the hash expressions are identical in all signatures, which makes aggregation possible.

![test](images/musig_intro_3.jpg)

#### 1.2.3 _Programming exercise:_ Compute aggregated MuSig signature

In this exercise, we'll create individual signatures and then aggregate them to create a valid signature.

Use the `sign_musig()` function to create individual signatures. `sign_musig()` takes:
  - the individual participant's private key (an `ECKey` object)
  - the invididual participant's nonce (an `ECKey` object)
  - the aggregate nonce point (an `ECPubKey` object)
  - the aggregate pubkey (an `ECPubKey` object)
  - the message (a 32 byte `bytes` object)

and returns an individual signature (a 64 byte `bytes` object containing `R(x)` and `s`).

Use `aggregate_musig_signatures()` to aggregate the individual signatures. `aggregate_musig_signatures()` takes a list of signatures and returns the aggregated signature.

Use `ECPubKey.verify_schnorr(sig, msg)` to verify that the signature is valid.

In [ ]:
msg = hashlib.sha256(b'transaction').digest()

# Generate individual signatures
sig0 =  # TODO: implement
sig1 =  # TODO: implement
sig2 =  # TODO: implement

# Aggregate signatures
sig_agg =  # TODO: implement

# Verify signature
assert pk_musig.verify_schnorr(sig_agg, msg)
print("Success!")